In [1]:
import pandas as pd
rating_df = pd.read_csv('ratings.csv')
rating_df

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0
...,...,...,...
233301,1540125,DS0101EN,3.0
233302,1250651,PY0101EN,3.0
233303,1003832,CB0105ENv1,3.0
233304,922065,BD0141EN,3.0


In [2]:
rating_df['item'].value_counts()

item
PY0101EN      14936
DS0101EN      14477
BD0101EN      13291
BD0111EN      10599
DA0101EN       8303
              ...  
ST0201EN          1
DX0106EN          1
ST0301EN          1
DX0108EN          1
GPXX0QR3EN        1
Name: count, Length: 126, dtype: int64

In [3]:
rating_df['user'].value_counts()

user
2          61
5          59
1653994    51
924030     51
746163     50
           ..
1058827     1
715501      1
763737      1
506416      1
711047      1
Name: count, Length: 33901, dtype: int64

In [4]:
df = rating_df.pivot(index='user', columns='item', values='rating')

# Replace NaN values with "?" to indicate missing ratings
# pivot_df.fillna("?", inplace=True)

# Display the final pivoted DataFrame
df

item,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,BD0131EN,...,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0101EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
user,,,,,,,,,,,,,,,,,,,,,
2,NaN,3.0,NaN,NaN,3.0,2.0,NaN,2.0,2.0,3.0,...,NaN,2.0,NaN,3.0,NaN,2.0,2.0,NaN,3.0,NaN
4,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,2.0,NaN,NaN,NaN,2.0,2.0,NaN,2.0,2.0
5,2.0,2.0,2.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102054,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2102356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2102680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from surprise import SVD, Dataset, Reader

In [6]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Melt the DataFrame to long format (Surprise requires user-item-rating columns)
long_df = df.reset_index().melt(id_vars=['user'], var_name='item', value_name='rating')

# Drop rows with missing ratings
long_df = long_df.dropna()

# Define the Reader for Surprise
reader = Reader(rating_scale=(1, 5))

# Load the data into Surprise format
data = Dataset.load_from_df(long_df[['user', 'item', 'rating']], reader)

# Train-test split (20% of data for testing)
trainset, testset = train_test_split(data, test_size=0.2)

# Use SVD (Singular Value Decomposition) for matrix factorization
model = SVD()

# Train the model on the training set
model.fit(trainset)

# Predict ratings for the test set
test_predictions = model.test(testset)

# Calculate RMSE for the test set predictions
rmse = accuracy.rmse(test_predictions)

# # Predict missing values for all users
# predictions = []
# for user in df.index:
#     user_predictions = []
#     for item in df.columns:
#         prediction = model.predict(user, item)
#         user_predictions.append(prediction.est)
#     predictions.append(user_predictions)

# # Create a DataFrame from the predictions and fill in missing values
# predicted_df = pd.DataFrame(predictions, columns=df.columns, index=df.index)

# # Replace missing values in the original DataFrame with predicted ones
# df_filled = df.copy()
# for user in df.index:
#     for item in df.columns:
#         if pd.isna(df_filled.loc[user, item]):
#             df_filled.loc[user, item] = predicted_df.loc[user, item]

RMSE: 0.2896
